In [1]:
!pip install tensorflow
!pip install matplotlib
!pip install numpy
!pip install keras
!pip install PIL
from PIL import Image 
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

     -------------------------------------- 272.8/272.8 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 4.1/4.1 MB 810.0 kB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 422.5/422.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 2.0 MB/s eta 0:00:00
     -------------------------------------- 440.7/440.7 kB 3.1 MB/s eta 0:00:00
     -----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.5 which is incompatible.


In [27]:
!mkdir Train
!mkdir Test

!mkdir Train/Fake
!mkdir Train/Real

!mkdir Test/Fake
!mkdir Test/Real


A subdirectory or file Train already exists.
A subdirectory or file Test already exists.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.


In [18]:
import os

os.makedirs("Train/Fake", exist_ok=True)
os.makedirs("Train/Real", exist_ok=True)
os.makedirs("Test/Fake", exist_ok=True)
os.makedirs("Test/Real", exist_ok=True)


In [19]:
# Train verilerini keras 'ın okuyabileceği şekilde konumlarını yeniden düzenliyorum.

PATH = "C:\\Users\\Hanife\\Desktop\\gg\\dataset\\train"

for i in os.listdir(PATH):
    control = i.split("_")
    
    try:
        if control[1] == "forg":
            os.system("cp -r {} Train\\Fake".format(PATH + "\\" + i))
    except:
        os.system("cp -r {} Train\\Real".format(PATH + "\\" + i))


In [28]:
# Test verilerini keras 'ın okuyabileceği şekilde konumlarını yeniden düzenliyorum.

PATH = "C:\\Users\\Hanife\\Desktop\\gg\\dataset\\test"

for i in os.listdir(PATH):
    
    contol = i.split("_")
    
    try:
    
        if contol[1]=="forg":
            
            os.system("cp -r {} Test/Fake".format(PATH+i))
            
    except:
        
        os.system("cp -r {} Test/Real".format(PATH+i))
        

In [29]:
# konumları

train_dir = os.path.join(".", "Train")
test_dir = os.path.join(".", "Test")


In [30]:
train_datagen = ImageDataGenerator(

      # resim pixellerini 0,1 arasına sıkıştırma
      rescale=1./255,

      # derece cinsinden (0-180) resimlerin rastgele döndürülme açısı
      rotation_range=40,

      # resimlerin yatayda ve dikeyde kaydırılma oranları
      width_shift_range=0.2,

      # resimlerin yatayda ve dikeyde kaydırılma oranları
      height_shift_range=0.2,

      # burkma işlemi
      shear_range=0.2,

      # yakınlaştırma işlemi
      zoom_range=0.2,

      # dikeyde resim döndürme
      horizontal_flip=True,

      # işlemlerden sonra ortaya çıkan  fazla 
      # görüntü noktalarının nasıl doldurulacağını belirler
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [31]:
train_generator = train_datagen.flow_from_directory(

        # hedef dizin
        train_dir,

        # tüm resimler (150x150) olarak boyutlandırılacak
        target_size=(200, 200),

        # yığın boyutu
        batch_size=64,

        # binary_crossentropy kullandığımız için
        # ikili etiketler gerekiyor.
        class_mode='binary')

Found 0 images belonging to 2 classes.


In [16]:
test_generator = test_datagen.flow_from_directory(

        test_dir,

        target_size=(200, 200),

        batch_size=64,

        class_mode='binary')

Found 0 images belonging to 0 classes.


In [ ]:
plt.imshow(train_generator[0][0][5])
print("Label : ",train_generator[0][1][5])

In [ ]:
plt.imshow(train_generator[0][0][60])
print("Label : ",train_generator[0][1][60])

In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    
    self.cnn1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(3,200,200))
    self.cnn2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
    self.cnn3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')
    
    self.flatten = tf.keras.layers.Flatten()
    
    self.dense1 = tf.keras.layers.Dense(512, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    
    x = self.cnn1(inputs)
    x = self.cnn2(x)
    x = self.cnn3(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

model = MyModel()

In [ ]:
input_shape = (None, 200, 200, 3)
model.build(input_shape)
model.summary()

In [ ]:
model.compile(
    # kayıp fonksiyonu
    loss="binary_crossentropy",
    
    # eniyileme:
    # ağımızın girdisi olan veri ile oluşturduğu kaybı göz önünde
    # bulundurarak kendisini güncelleme mekanizması
    optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),

    # eğitim ve test süresince takip edilecek metrikler. 
    metrics=["acc"])

In [ ]:
# acc, loss, val_acc, val_loss değerlerini history adlı değişkenden alacağız.
history = model.fit_generator(

    # eğitim verileri
    train_generator,

    # döngü bitene kadar geçeceği örnek sayısı (alınacak yığın)
    steps_per_epoch=train_generator.samples//train_generator.batch_size,

    # döngü sayısı
    epochs=40,

    verbose=2)


In [ ]:
# Eğitim başarım skoru
acc = history.history["acc"]

# eğitim kayıp skoru
loss = history.history["loss"]

# epochs sayısına göre grafik çizdireceğiz.
epochs = range(1, len(acc) + 1)

# eğitim başarımını kendine özel çizdirdik.
plt.plot(epochs, acc, "bo", label="Eğitim başarımı")

# çizdirmemizin başlığı
plt.title("Eğitim başarımı")

plt.legend()

plt.figure()

# eğitim kaybını kendine özel çizdirdik.
plt.plot(epochs, loss, "bo", label="Eğitim kaybı")


# çizdirmemizin başlığı
plt.title("Eğitim kaybı")

plt.legend()

# ekrana çıkartma
plt.show()

In [ ]:
# Test verileri ile kayip ve doğrulama 
model.evaluate(test_generator)

In [ ]:
plt.imshow(test_generator[0][0][5])
print("Label : ",test_generator[0][1][5])

test_input = test_generator[0][0][5]
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)

pred = model.predict(test_input)

if pred>=0.5:
    pred = 1
    
else:
    
    pred = 0
    
print("Predict : ",float(pred))

In [ ]:
plt.imshow(test_generator[0][0][30])
print("Label : ",test_generator[0][1][30])

test_input = test_generator[0][0][30]
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)
test_input = np.expand_dims(test_input,axis=0)

pred = model.predict(test_input)

if pred>=0.5:
    pred = 1
    
else:
    
    pred = 0
    
print("Predict : ",float(pred))